# Importing Libraries and Exploring Data

In [ ]:
#import the libraries
import numpy as np
import pandas as pd

patient_data=pd.read_csv("Health_Data.csv")
#use the head fucntion to get a glimpse data
patient_data.head()

In [ ]:
# Summary of Numerical Data
(patient_data.describe())

#Summary of Categotical Data
patient_data.describe(include=['object'])

In [ ]:
#Summary of Categotical Data
patient_data.describe(include=['object'])

In [ ]:
# Summary of Numerical Data
(patient_data.describe())

In [ ]:
#Seperate independent and dependent variables
mydata=pd.read_csv("Health_Data.csv")
X=mydata.iloc[:,1:9]
y=mydata.iloc[:,9]

#Datatye of independent and dependent variable
print("X is:",type(X))
print("y is:",type(y))

In [ ]:
X.head()


In [ ]:
y.head()

In [ ]:
print("X is:",type(X))
print("y is:",type(y))

In [ ]:
X.head()

# Transforming Categorical Variables

In [ ]:
# New Admission type
A_type=pd.get_dummies(X.iloc[:,1],drop_first=True,prefix='Atype')

# New Gender  
New_gender=pd.get_dummies(X.iloc[:,4],drop_first=True,prefix='Gender')

# New Pre Existing Disease Variable
Pre_exdis=pd.get_dummies(X.iloc[:,2],drop_first=True,prefix='PreExistDis')


In [ ]:
#Drop the Original columns
X.drop(['Admission_type','PreExistingDisease','Gender'],axis=1,inplace=True)

# Concat the new transformed data to X dataframe
X=pd.concat([X,A_type,New_gender,Pre_exdis],axis=1)

In [ ]:
X.head()

# Splitting Train Test Data & Scaling it

In [ ]:
# Split The Data into Train and Test
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest= train_test_split(X, y, test_size=0.30, random_state=110)

In [ ]:
#Scale the inputs
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

xtrain=sc.fit_transform(xtrain)
xtrain=pd.DataFrame(xtrain,columns=xtest.columns)

xtest=sc.transform(xtest)
xtest=pd.DataFrame(xtest,columns=xtrain.columns)

# Create new objects to store the values as Numpy arrays
x_train=xtrain.values
x_test=xtest.values
y_train=ytrain.values
y_test=ytest.values

In [ ]:
xtrain.head()

In [ ]:
x_train=xtrain.values
x_test=xtest.values
y_train=ytrain.values
y_test=ytest.values

# Building an Arificial Neural Network & Training it


In [ ]:
#Import the relevant Keras libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

# Initiate the Model with Sequestial Class
model=Sequential()



In [ ]:
# Add the 1st dense layer and Dropout Layer
model.add(Dense(units=6,activation='relu',kernel_initializer='uniform',input_dim=11))
model.add(Dropout(rate=0.3))

#Add the 2nd dense Layer and Dropout Layer
model.add(Dense(units=6,activation='relu',kernel_initializer='uniform'))
model.add(Dropout(rate=0.3))

# Add Output Dense Layer
model.add(Dense(units=1,activation='sigmoid',kernel_initializer='uniform'))

In [ ]:
#Complie the Network
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

#Fit the Model
model.fit(x_train,y_train,epochs=100,batch_size=20)


In [ ]:
# Create 2 prediction variables y_pred_class and y_pred_prob
# y_pred_class is the predcition & y_pred_prob is probabilities of the prediction
y_pred_class=model.predict(x_test)
y_pred_prob=model.predict_proba(x_test)


In [ ]:
y_pred_class[:5]

# Accuracy & Null Accuracy

In [ ]:
#Set threshold for 0 and 1. all values above threshold are 1 and below 0 
y_pred_class=y_pred_class>0.5


# false mean 0 and true mean 1
print(y_pred_class[:5])


In [ ]:
y_pred_class.astype(int)[:5]

In [ ]:
# Calculate the accuracy score using ScikitLearn
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred_class)

In [ ]:
# Use the Panda Series value_count function to calculate distinct class values 
ytest.value_counts()

In [ ]:
#use head function and divide it by lenght of ytest
ytest.value_counts().head(1)/len(ytest)

In [ ]:
y.value_counts().head(1)/len(y)

# Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
# Always remember that the original values (y_test) should be passed as 1st parameter and y_pred as 2nd
cm=confusion_matrix(y_test,y_pred_class)
cm

# Sensitivity, Specificity, Precision & FPR

In [ ]:
#True Negative
TN=cm[0,0]

#False Negative
FN=cm[1,0]

#False Positives
FP=cm[0,1]

#True Positives
TP=cm[1,1]

In [ ]:
# Manually Calculating Sensitivity
Sensitivity=TP/(TP+FN)
Sensitivity


In [ ]:
# Calculating Sensitivity with inbuild function
from sklearn.metrics import recall_score
recall_score(y_test,y_pred_class)


In [ ]:
#Calculating Specificity
Specificity=TN/(TN+FP)
Specificity

In [ ]:
#Precision
Precision= TP/(TP+FP)
Precision

In [ ]:
#Calculate False Positive_rate
Positive_rate= FP/(FP+TN)
Positive_rate

# Working with ROC, AUC & Thresholds

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#histogram of class distribution
plt.hist(y_pred_prob)
plt.title("Histogram of Predicted Probabilites")
plt.xlabel("Predicted Probabilities of patient readmisson")
plt.ylabel("Frequency")
plt.show()

In [ ]:
from sklearn.metrics import roc_curve
fpr,tpr,thresholds=roc_curve(y_test,y_pred_prob)


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(fpr,tpr)
plt.title("ROC Curve for Patient Readmission")
plt.xlabel("False Positive rate")
plt.ylabel("True Positive rate")
plt.show()

In [ ]:
#ROC curve
from sklearn.metrics import roc_curve
fpr,tpr,thresholds=roc_curve(y_test,y_pred_prob)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(fpr,tpr)
plt.title("ROC Curve for Patient Readmission")
plt.xlabel("False Positive rate (1-Specificity)")
plt.ylabel("True Positive rate (Sensitivity)")
plt.grid(True)
plt.show()

In [ ]:
def optimum_threshold(my_threshold):
    print ("Sensitivity:",tpr[thresholds>my_threshold][-1])
    print ("Specificity:",1-fpr[thresholds>my_threshold][-1])

In [ ]:
optimum_threshold(0.3)

In [ ]:
optimum_threshold(0.03)

In [ ]:
#Auc Score
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,y_pred_prob)